In [1]:
import sys
import pandas as pd
import modeles

# Chargement des hélico

In [2]:
helicos = pd.read_excel("Data/Copie de 20210108_161349_end_product_export_administrator.xlsx")
maintenances = pd.read_excel("Data/20210105_Historique maintenance.xlsx", sheet_name="HISTORIQUE PREV_CORR",
    names=["intervention", "avion", "type", "temps", "date", "statut"], index_col=0, parse_dates=["date"])
helicos.sample(5)
maintenances.sample(5)

,avion,type,temps,date,statut
intervention,,,,,
2017-00628590,000G2369,CORRECTIVE,32.5,2017-04-01,CLOTUREE
2019-00056060,000G2331,CORRECTIVE,4.0,2019-02-05,CLOTUREE
2019-00061676,000G2272,PREVENTIVE,0.0,2019-02-07,DIFFEREE_NORMAL
2019-00033798,000G2325,CORRECTIVE,2.0,2019-01-23,CLOTUREE
2020-00001255,000G2331,CORRECTIVE,73.0,2020-01-06,CLOTUREE


In [3]:
print(f"Il y a {len(helicos)} disponibles et {len(helicos[helicos['state'] == 'SERVICEABLE'])} prêts")

def historique_maintenances_appareil(nom:str):
    return maintenances[maintenances["avion"] == nom]

historique = historique_maintenances_appareil("000G2342")
print(historique.head(10))
print(f"{len(historique[historique['statut'] == 'ACTIVE'])} maintenances en cours")

Il y a 26 disponibles et 14 prêts
                  avion        type  temps       date    statut
intervention                                                   
2017-00570315  000G2342  CORRECTIVE    0.0 2017-02-22   ANNULEE
2017-00570333  000G2342  CORRECTIVE    8.0 2017-02-22  CLOTUREE
2017-00570345  000G2342  PREVENTIVE  411.0 2017-02-22  CLOTUREE
2017-00626666  000G2342  CORRECTIVE   19.0 2017-03-30  CLOTUREE
2017-00627155  000G2342  PREVENTIVE    4.5 2017-03-31  CLOTUREE
2017-00631570  000G2342  PREVENTIVE    0.0 2017-04-04   ANNULEE
2017-00681954  000G2342  CORRECTIVE  326.0 2017-05-11  CLOTUREE
2017-00683184  000G2342  PREVENTIVE    1.0 2017-05-11  CLOTUREE
2017-00686222  000G2342  CORRECTIVE    0.0 2017-05-15  CLOTUREE
2017-00697130  000G2342  CORRECTIVE   44.0 2017-05-22  CLOTUREE
15 maintenances en cours


# Panne de l'appareil
Probabilité de panne des pièces dans les prochaines 24h en fonction de leur âge et de leur espérance de vie.
Utilise la loi normale pour la répartition des _morts_ dans le temps.

$P\left[panne \in [t, t+dt]\right] = 0.5*\left(erf\left(\frac{t + dt - MTBF}{\sigma \sqrt{2}}\right) - erf\left(\frac{t - MTBF}{\sigma \sqrt{2}}\right)\right)$

In [4]:
import math
VAR = 0.3

mtbf = pd.read_excel("Data/20210105_MTBF vs MTBUR.xlsx",
        names=["index", "MTBF", "MTBUR"],
        index_col=0, usecols=[0, 4, 5], converters={"MTBF":int, "MTBUR":int})
print(mtbf.sample(n=5))

def probabilite_panne_helico(helico_id) -> float:
    pieces = pd.read_excel(f"Data/Planes/end_product_{helico_id}_at_20210107.ods",
        sheet_name="in service parts history",
        index_col=[0], usecols=[0, 3, 4, 7, 8], parse_dates=[3, 4], names=["element id", "part number", "serial number", "start", "end"])
    utilisations = pd.read_excel(f"Data/Planes/end_product_{helico_id}_at_20210107.ods",
        sheet_name="life records", usecols=[1, 2, 5], index_col=[0, 1])
    if len(utilisations) == 0:
        print("Aucun usage enregistré de", helico_id)
        return None
    
    j = pieces.join(mtbf, how="left").join(utilisations, on=("part number", "serial number"), how="inner")
    #print(j.sample(n=5))
    
    dt = 24
    defaillances = j.assign(defect_20h=0.5 * (((j.value + dt - j.MTBF) / (VAR * j.MTBF * math.sqrt(2))).apply(math.erf) -\
                                              ((j.value - j.MTBF) / (VAR * j.MTBF * math.sqrt(2))).apply(math.erf)))
    return defaillances
    
for h_id in ["EPR0000020","EPR0000027", "EPR0000028"]:
    def_20 = probabilite_panne_helico(h_id)
    if def_20 is not None:
        print(def_20.sample(10))
        def_20.to_csv(f"defaillance_{h_id}.csv")

           MTBF MTBUR
index                
321201-01  1051   620
6323_01     341   201
94670      1242   732
2250363    4294  2533
346000-01  9556  5638
          part number serial number      start end  MTBF MTBUR    value  \
475001      145754546          1240 2019-01-01 NaT   955   563    329.2   
94684       144181965           922 2019-01-01 NaT  1242   732    281.7   
6510_03     144181869          1013 2019-01-01 NaT  3284  1937    373.0   
94676       144181966          1434 2019-01-01 NaT  1242   732    371.7   
94676       144181966          1434 2019-01-01 NaT  1242   732    371.7   
94684       144181965           922 2019-01-01 NaT  1242   732    281.7   
62103       144352864          1249 2019-01-01 NaT   363   214    162.6   
2341-03     144315362           526 2019-01-01 NaT  2389  1409    121.2   
22301       145754555           771 2019-01-01 NaT   484   285    351.2   
321201-02   144023747           731 2019-01-01 NaT  1051   620  99605.8   

           defect_20